<h1 style="color:black" align="center">Нормализации</h1>

<h1 style="color:#008B8B">1. Идея нормализации</h1>

### Covariate shift

В машинном обучении существует термин - Covariate shift, который обозначает сдвиг области определения. Имеется некоторая обучающая выборка с двумя классами, которая устроена следующим образом:

<img src='img/lecture05/1.png' width="500" height="500">

Исходный набор данных модель способна отделить прямой синего цвета. После, на тестовой выборке, которую мы собрали позже, распределение признаков изменилось (немного съехали под другим углом). Тогда, синяя модель будет плохо работать на тестовых данных и тут будет хорошей моделью красная линия.

* В классическом машинном обучении — изменение распределения данных
* Много методов решения

### Domain adaptation

Имеется обучающая и тестовая выборка. Предположим у нас имеются тестовые объекты, но мы не имеем значение целевой переменной. 

* Объекты по-разному распределены на обучении и на контроле
* Идея: взвешивать объекты при обучении

Тогда при обучении в функционал вставим веса $s_i$ при каждом объекте $x_i$ и этот вес будет большим, если $x_i$ объект похож на объект из тестовой выборки, а если объект не похож на тестовый, тогда вес будет маленьким:

$$\large \sum\limits_{i=1}^{\ell} s_i (a(x_i) - y_i)^2 \to \underset{a(x)}{\text{min}} $$

* Большие веса будем ставить объектам, которые похожи на объекты из тестовой выборки

Да, при обучении мы смотрим на тестовоую выборку, но мы используем только признаки тестовой выборки не используя целевую переменную.

### Internal covariate shift

* В нейронной сети каждый слой обучается на выходах предыдущих слоёв
* Если слой в начале сильно меняется, то все следующие слои надо переделывать

Имеется некоторая нейронная сеть:

<img src='img/lecture05/2.png' width="500" height="500">

Предположим, что градиентный спуск в процессе и мы находимся на 100-й итерации (одна итерация - проход по батчу), следовательно, мы имеем уже некоторые параметры. 

Cейчас, на выходе первого полносвязно слоя имеются числа (1, 2, 0, 1) и веса при следующем слое еденичные. Предположим, мы делаем следующий градиентный шаг и в результате очередной итерации градиентного спуска - сильно изменились веса первого полносвязного слоя. Из-за этого сильно изменились выходы первого полносвязного слоя:

<img src='img/lecture05/3.png' width="500" height="500">

Из-за такого изменения последующие слои потеряли смысл, так как они приспособлены к другому масштабу около еденицы. Последующие слои обучались на более меньший масштаб и из-за данного изменения последующие слои будут плохо работать. Следовательно, если сильно меняется изначальный слой, тогда все последующие необходимо перестраивать.

* Идея: преобразовывать выходы слоёв так, чтобы они гарантированно имели фиксированное распределение

<h1 style="color:#008B8B">2. Batch Normalization</h1>

* Реализуется как отдельный слой
* Вычисляется для текущего батча

Нормализация вставляется как ещё один слой. Имеется некоторый $FC$ слой, выходы которго подаются на вход Batch Norm Layer и он выдает вектор такой же длины как входной.

Рассмотрим подробнее как это работает? Мы обучаем модель батчами, имеется некоторая начальная сеть, через которую мы прогоняем батч и на выходе получаем для каждого объекта свой вектор. Эти выходы мы подаем на вход батч-норм-слою, который по каждой компоненте считает статистики. Если например, векторы размера 200, тогда и $\mu_B$ будет размера 200.

* Оценим среднее и дисперсию каждой компоненты входного вектора:

$$\large \mu_{B} = \frac{1}{n} \sum\limits_{j=1}^{n} x_{B, j}
\qquad
\sigma_{B}^2 = \frac{1}{n} \sum\limits_{j=1}^{n} (x_{B, j} - \mu_{B})^2$$

Где $x_{B,j}$ - $j$-й объект в батче $B$

Следовательно мы получам вектор среднего и вектор дисперсий. 

* Отмасштабируем все выходы:

$$\large \tilde{x}_{B, j} = \frac{x_{B,j} - \mu_{B}}{\sqrt{\sigma^2_{B} + \epsilon}}$$

Это ограничивает возможности нейронной сети, так как мы требуем, что после каждого слоя, каждый нейрон имеет нулевое среднее и еденичное стандартное отклонение. 

Поэтому, добавим парамтров в нейронную сеть. После того, как мы выполнили стандартизацию, у каждого $j$-го выхода предыдущего слоя среднее $\beta$ и стандартное отклонение $\gamma$. Для этого зададим нужные нам среднее и дисперсию следующим образом:

$$\large z_{B, j} = \gamma_j \circ \tilde{x}_{B,j} + \beta_j$$

После этого, получится что $j$-й выход имеет среднее $\beta$ и стандартное отклонение $\gamma$, где $\beta, \gamma$ - это обучаемы параметры.

**Итого:** Батч-норм-слой гарантирует, что после прохождения батча через него, по каждой координате свое фиксированное среднее и дисперсия.

**Этап применения:**

Во время применения нейронной сети:
* Те же самые формулы, но вместо $\mu_{B}$ и $\sigma_{B}^2$ используем их средние значения по всем батчам, так как нейронная сеть применяется к отедльным объектам, а не к батчам.

**Как использовать?**

* Обычно вставляется между полносвязным/свёрточным слоём и нелинейностью
* Использование Batch Normalization позволяет увеличить длину шага в градиентном спуске, что ускоряет обучение.
* Не факт, что действительно устраняет covariance shift. Но если наши предположения были не верны и мы не решим проблему covariance shift, тогда использовать Batch Normalization стоит, так как эксперементы показывают, что Batch Normalization улучшает качество обучения нейронной сети.

## 2.1 В чём польза от BatchNorm?

Статья была устроена так, выдвинули гипотезу, проверили - работает! Но соврешенно не понятно, верна ли гипотеза? Верно ли что слои могу резко измениться, верны ли все домыслы? 

После выхода статьи про Batch Normalization, вышла новая статья под названием $\text{How does batch normalization help optimiztion?}$. В статье провели ряд эксерементов. 

### Гипотеза 1

Если Batch Normalization помогает стабилизировать распределение выходов, спасая от резких изенений (за счет того, что распределение выходов не изменится) из-за которых необходимо престраивать все последующие веса нейронной сети. 

Тогда, возьмем нейронную сеть, которая хорошо обучается с использованием BatchNorm. Известно, что BatchNorm выдает векторы со стабильным распределением, известным средним и дисперсией. Добавим шума вектор, который выдет BatchNorm, испортив тем самым распределение выхода BatchNorm после каждого шага.

<img src='img/lecture05/6.png' width="700" height="700">

Оказывается, это ничего не портит, нейронная сеть все равно хорошо обучается, аналогично тому, как обучение без добавления шума. Тогда, логично, что BatchNorm не связан с выравниванием распределения слоёв. 

### Гипотеза 2

Была проблема с градиентами. Если веса в начале нейронной сети изменяются сильно, тогда придется сильно перестраиваться веса при следующих слоях. А мы выдвинули гипотезу, что BatchNorm должен исправлять эту проблему. И если провести сравнение градиентов по разным слоям между итерациями, окажется, что при использовании BatchNorm градиенты так же сильно изменяются: 

<img src='img/lecture05/7.png' width="700" height="700">

Следовательно, когда используется BatchNorm, иногда, в процессе обучении сильно изменяются веса во всех слоях. Делаем выводы, что BatchNorm не решает проблему сильного перестраивания дальнейших слоев при изменении ранних слоев.

### Польза от BatchNorm

Оказывается, что при добавлении BatchNorm в нейронную сеть делат процесс оптимизации более простой, за счет того, что функционал ошибки становится более гладким. И после добавления BatchNorm обучение становится более стабильным:

<img src='img/lecture05/8.png' width="700" height="700">